In [23]:
import tensorflow as tf

In [24]:
import matplotlib.pylab as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [25]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [26]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
IMAGE_RES = 224
feature_extractor = hub.KerasLayer(URL, input_shape=(IMAGE_RES, IMAGE_RES, 3))

In [28]:
# Loading dataset from the URL
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    with_info=True,
    as_supervised=True,
    split=['train[:80%]', 'train[80%:]'],
)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

2023-10-07 01:59:58.381745: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]WARNING:absl:1738 images were corrupted and were skipped
                                                                        

Dataset cats_vs_dogs downloaded and prepared to /Users/quangnguyen/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [29]:
# Reformat images to resolution (224, 224)
def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    return image, label

BATCH_SIZE = 32

train_batches = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [30]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

In [31]:
feature_batch = feature_extractor(image_batch)
print(feature_batch.shape)

(32, 1280)


In [32]:
feature_extractor.trainable = False

In [33]:
model = tf.keras.Sequential([
    feature_extractor,
    layers.Dense(2)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [35]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

EPOCHS = 6
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/6
 12/582 [..............................] - ETA: 4:24 - loss: 0.0466 - accuracy: 0.9818

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


 44/582 [=>............................] - ETA: 4:30 - loss: 0.0319 - accuracy: 0.9872

 54/582 [=>............................] - ETA: 4:21 - loss: 0.0284 - accuracy: 0.9896

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


...

391/582 [===================>..........] - ETA: 1:25 - loss: 0.0329 - accuracy: 0.9882

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


582/582 [==============================] - ETA: 0s - loss: 0.0312 - accuracy: 0.9894

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9


KeyboardInterrupt: 